#  WEB SCRAPING AND DATA CLEANING OF Top 200 Movies at Worldwide Box Office from 2000 to 2017
 * acquire the data from the box office Mojo 
 * after data cleaning, save it into boxoffice.csv file

In this part, first we did webscraping to extract the yearly box office data. Then we did data cleaning.


### The result of this Step is a CSV file:

* 'boxoffice.csv'

In [1]:
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
import time
import re

In [2]:
def scrap(year):
    '''
    scarp yearly box office data from boxofficemojo
    input: year, integer number
    output: dataframe of yearly box office information
            colnames=[['Year','Rank', 'Title', 'Studio','Worldwide','Domestic','Overseas','Category','Link']]
            'Link': the movie's web page at mojo
    '''
    url = 'http://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr='+str(year)
    html = urllib.request.urlopen(url)    
    bsObj = BeautifulSoup(html,'lxml-xml')
    
    href_list=bsObj.findAll('a', attrs={'href': re.compile(r"movies")})
    
    Link=[]
    for i in range(1,201):
        m=href_list[i].attrs
        x=str(list(m.values())[0])
        y="http://www.boxofficemojo.com"+x
        z=y.replace('id',r'page=intl&country=CH&id')
        Link.append(z)
        
    td_list=bsObj.find_all('td') 
    data=[]
    for td in td_list:
        l=[]
        for tr in td:
            l.append(tr.string)
        data.append(l)
    
    I = list(range(0,10))
    data=np.delete(data, I).tolist()
    
    Rank=[]
    Title=[]
    Studio=[]
    Worldwide=[]
    Domestic=[]
    Overseas=[]
    for i in range(1599):
        col=i%8
        x=data[i][0]
        if col==0:
            Rank.append(x)
        elif col==1:
            Title.append(x)
        elif col==2:
            Studio.append(x)
        elif col==3:
            Worldwide.append(x)
        elif col==4:
            Domestic.append(x)
        elif col==6:
            Overseas.append(x)
            
    dict={'Rank':Rank,
          'Title':Title,
          'Studio':Studio,
          'Worldwide':Worldwide,
          'Domestic':Domestic,
          'Overseas':Overseas,
          'Link':Link}

    boxoffice = pd.DataFrame(dict)                   
    
    row=boxoffice.shape[0]
    Category=[]
    for i in range(50):
        Category.append("Top1to50")
    for i in range(50,100):
        Category.append("Top51to100")
    for i in range(100,150):
        Category.append("Top101to150")
    for i in range(150,200):
        Category.append("Top151to200")
    boxoffice['Category']=Category
    
    Year=[]
    for i in range(200):
        Year.append(year)
    boxoffice['Year']=Year
    
    boxoffice = boxoffice[['Year','Rank', 'Title', 'Studio','Worldwide','Domestic','Overseas','Category','Link']]
    
    return boxoffice

In [3]:
def clean(year):
    '''
    clean data 
    input: year, integer number
    output: a tidy dataframe, in which values of the box office are numeric
    '''
    boxoffice=scrap(year)
    row=boxoffice.shape[0]
    
    def Replace(colname):
        '''
        Replace','and '$',and 'k'
        '''

        for i in range(row):
            x=boxoffice[colname][i]
            y=x.replace(',','') 
            z=y.replace('$','')
            boxoffice[colname][i]=z
            if z!="n/a":
                if z.endswith("k"):
                    replace=z.replace('k','')
                    boxoffice[colname][i]=float(float(replace)*0.0100)
        return
    
    Replace("Domestic")
    Replace("Worldwide")
    Replace("Overseas")

    
    #Make sure all columns of numbers are numeric (not strings).
    boxoffice.Worldwide =pd.to_numeric(boxoffice.Worldwide, errors='coerce').fillna(0).astype(float)
    boxoffice.Domestic =pd.to_numeric(boxoffice.Domestic, errors='coerce').fillna(0).astype(float)
    boxoffice.Overseas =pd.to_numeric(boxoffice.Overseas,errors='coerce').fillna(0).astype(float) 
     
    data=boxoffice.round({'Worldwide': 1, 'Domestic': 1,'Overseas':1})
       
    return data

In [4]:
df=clean(2000)
for i in range(2001,2018):
    i=str(i)
    data=clean(i)
    df=df.append(data, ignore_index=True)

C:\Users\qxzhao\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\qxzhao\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
df.to_csv('boxoffice.csv',index=False)

In [6]:
df

,Year,Rank,Title,Studio,Worldwide,Domestic,Overseas,Category,Link
0,2000,1,Mission: Impossible II,Par.,546.4,215.4,331.0,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
1,2000,2,Gladiator,DW,457.6,187.7,269.9,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
2,2000,3,Cast Away,Fox,429.6,233.6,196.0,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
3,2000,4,What Women Want,Par.,374.1,182.8,191.3,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
4,2000,5,Dinosaur,BV,349.8,137.7,212.1,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
5,2000,6,How the Grinch Stole Christmas,Uni.,345.1,260.0,85.1,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
6,2000,7,Meet the Parents,Uni.,330.4,166.2,164.2,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
7,2000,8,The Perfect Storm,WB,328.7,182.6,146.1,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
8,2000,9,X-Men,Fox,296.3,157.3,139.0,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
9,2000,10,What Lies Beneath,DW,291.4,155.5,136.0,Top1to50,http://www.boxofficemojo.com/movies/?page=intl...
